In [2]:
# install once in your environment:
# pip install ultralytics opencv-python
from ultralytics import YOLO
import cv2
import numpy as np
import mss
import torch
import seaborn
from PIL import Image
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
# Load YOLOv8 model on GPU
model = YOLO("model/yolov8x.pt").to(device)
# 1. Load a pretrained YOLOv8 model
model = YOLO("model/yolov8l.pt")   # "n" = nano (fast); you can try "s", "m", "l" for larger


Using device: cuda


In [ ]:
# Run prediction on an image
results = model("TestFootage\Frames\cartestframe10.jpg", show=True)  # show=True opens a window with boxes

# Print detailed results
for result in results:
    boxes = result.boxes
    for box in boxes:
        print(f"Class: {model.names[int(box.cls)]}, "
              f"Conf: {float(box.conf):.2f}, "
              f"Box: {box.xyxy.tolist()}")

FileNotFoundError: test.jpg does not exist

In [ ]:

#Run inference on an image
results = model("TestFootage\Frames\cartestframe10.jpg")   # path to your image

#Print detected classes and confidence
for result in results:
    boxes = result.boxes
    for box in boxes:
        print(f"Class: {model.names[int(box.cls)]}, "
              f"Conf: {float(box.conf):.2f}, "
              f"Box: {box.xyxy.tolist()}")



<>:11: SyntaxWarning: invalid escape sequence '\F'
<>:11: SyntaxWarning: invalid escape sequence '\F'
C:\Users\dukee\AppData\Local\Temp\ipykernel_28592\2931631625.py:11: SyntaxWarning: invalid escape sequence '\F'
  results = model("TestFootage\Frames\cartestframe10.jpg")   # path to your image


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\dukee\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

image 1/1 c:\Users\dukee\Documents\GitHub\ImageCaptioner\TestFootage\Frames\cartestframe10.jpg: 384x640 1 car, 171.2ms
Speed: 3.6ms preprocess, 171.2ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
car: 0.91


In [ ]:
# Set up screen capture
sct = mss.mss()
monitor = sct.monitors[1]  # 1 = primary screen, [0] = virtual all

while True:
    # Grab screen
    screenshot = sct.grab(monitor)
    frame = np.array(screenshot)

    # Convert BGRA -> BGR
    frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2BGR)

    # Run YOLO inference
    results = model(frame)

    # Draw only people (class 0 in COCO)
    for r in results:
        for box in r.boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            if cls == 0 and conf > 0.5:  # class 0 = person
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"Person {conf:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Show window
    cv2.imshow("Screen Capture - People Detection", frame)

    # Exit with 'q'
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cv2.destroyAllWindows()



0: 384x640 1 tv, 284.8ms
Speed: 5.1ms preprocess, 284.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 tv, 280.0ms
Speed: 2.0ms preprocess, 280.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 tv, 273.1ms
Speed: 1.5ms preprocess, 273.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 tv, 252.2ms
Speed: 1.5ms preprocess, 252.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 tv, 268.2ms
Speed: 1.8ms preprocess, 268.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 tv, 1 laptop, 277.1ms
Speed: 1.7ms preprocess, 277.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 tv, 1 laptop, 260.8ms
Speed: 1.8ms preprocess, 260.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 tv, 267.3ms
Speed: 1.6ms preprocess, 267.3ms inference, 2.8ms postprocess per image at shape (1, 3

In [3]:
# Setup screen capture
sct = mss.mss()
monitor = sct.monitors[1]  # full screen (use [0] for all monitors, or [2] for second monitor)

while True:
    # Capture screen
    screenshot = sct.grab(monitor)
    img = Image.frombytes("RGB", screenshot.size, screenshot.bgra, "raw", "BGRX")

    # Convert to numpy
    frame = np.array(img)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # Run YOLOv8 inference
    results = model(frame)

    # Draw detections
    annotated_frame = results[0].plot()

    # Show on screen
    cv2.imshow("YOLOv8 Screen Detection", annotated_frame)

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cv2.destroyAllWindows()



0: 384x640 2 tvs, 72.5ms
Speed: 24.8ms preprocess, 72.5ms inference, 69.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 tvs, 13.7ms
Speed: 1.4ms preprocess, 13.7ms inference, 3.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 tvs, 14.0ms
Speed: 1.4ms preprocess, 14.0ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 tvs, 26.1ms
Speed: 1.5ms preprocess, 26.1ms inference, 6.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 tvs, 14.6ms
Speed: 1.4ms preprocess, 14.6ms inference, 3.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 tvs, 14.3ms
Speed: 1.4ms preprocess, 14.3ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 tvs, 14.2ms
Speed: 1.4ms preprocess, 14.2ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 tvs, 14.3ms
Speed: 1.4ms preprocess, 14.3ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 